In [1]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()

In [2]:
import pandas as pd
import numpy as np
import os
import sys
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import tqdm

In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras import backend as K
import cv2

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from keras.optimizers import Adam

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Permute
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import GRU, LSTM

In [5]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
# Graph
from py2neo import Graph

# 1. Input data

In [7]:
PATH_DATA = "/Users/david/TFM_DATA/spec"
data_files = os.listdir(PATH_DATA)

In [8]:
# PATH triplets
PATH_TRIPLETS = os.path.join("..","triplets", "triplets.csv")
df = pd.read_csv(PATH_TRIPLETS, delimiter= ";")

In [9]:
# List
triplets_input = list(df["output"])
size_triplets_input = len(triplets_input)

# Sample
triplets_input_v1 = triplets_input[:1000]
size_triplets_input = len(triplets_input_v1)

# Artists
artists_labels = np.sort(df.a1.unique())
df_artists = pd.DataFrame(data = artists_labels, columns = ["artist"]).reset_index()
df_artists.columns = ["id","artist"]

# DF Artists with artists as index
df_artists_index = df_artists.set_index("artist")
num_artists = df_artists.shape[0]

### Catalan music

In [10]:
graph = Graph(bolt = True, host = "localhost", name = "Spotify", user = "neo4j", password = "qrks")

In [11]:
query = """
        MATCH (g:Genre)-[:GEN_ART]->(a:Artist)
        MATCH (a:Artist)-[:REL_ART]->(a2:Artist)
        WHERE toLower(g.genre_id) =~ '.*catala.*'
        RETURN a.artist_id, a2.artist_id

"""

In [12]:
cursor = graph.run(query)
dfneo = pd.DataFrame.from_records(cursor, columns=cursor.keys())

In [15]:
dfneo.to_csv("dfneo_CAT.csv", sep = ";", index = False)

In [13]:
set_art_cat = set()
for ii, row in dfneo.iterrows():
    set_art_cat.add(row["a.artist_id"])
    set_art_cat.add(row["a2.artist_id"])

In [14]:
#TODO: select couple of artists
sel_art = list(set_art_cat)

In [15]:
df_artists_cat = df_artists[df_artists.artist.isin(set_art_cat)]["artist"].reset_index()

In [16]:
df_artists_cat["id"] = df_artists_cat.index
df_artists_cat.drop('index', axis=1, inplace = True)

## Labels 

In [17]:
df_artists_gen = df_artists_cat.copy()

In [18]:
#labels_mat = keras.utils.to_categorical(df_artists_index.id)
labels_mat = keras.utils.to_categorical(df_artists_gen.id)

In [19]:
labels = dict()
for i, row in df_artists_gen.iterrows():
    labels[row.artist] = labels_mat[row.id,].astype(int)

In [20]:
num_artists = len(labels.keys())

## Songs

In [21]:
df_gen = df[df.a1.isin(set_art_cat)]

## 1.1 Dataframe Classiset_art_cat

In [22]:
df1 = df_gen[["a1","tr1","win1","ini1","fin1"]].copy()
df2 = df_gen[["a1","tr2","win2","ini2","fin2"]].copy()

# Column names
colnames = ["art","tr","win","ini","fin"]
df1.columns = colnames
df2.columns = colnames

In [23]:
df_concat = pd.concat([df1, df2])
df_concat.drop_duplicates(inplace = True)

In [24]:
df_concat["tr"] = df_concat.tr + "__" + df_concat.win.astype(str) +  \
    "__" + df_concat.ini.astype(str) + "__" + df_concat.fin.astype(str) + ".jpg"

In [26]:
df_concat.shape

(1915, 5)

In [27]:
df_concat = df_concat.reset_index().drop("index", axis = 1)

## 1.2 Data Generator

In [28]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, df, scaling, x_col, y_col=None, batch_size=10, num_classes=None, shuffle=True):
        self.batch_size = batch_size
        self.df = df
        self.indices = self.df.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.x_col = x_col
        self.y_col = y_col
        self.dim = (int(x_col / scaling), int(y_col / scaling), 1) #one input channel
        self.on_epoch_end()

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        idx = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in idx]
        
        X, y = self.__get_data(batch)
        return X, y

    def on_epoch_end(self):
        pass

    def __get_data(self, batch):
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size, self.num_classes), dtype=int)
        
        # Get the list of image files and corresponding artists
        df_imgs_files = self.df.iloc[batch]
        
        list_imgs = list(df_imgs_files["tr"])
        list_art = list(df_imgs_files["art"])
        
        for ii in range(len(list_imgs)):
            
            # Read image using cv2
            path_img = os.path.join(PATH_DATA, list_imgs[ii])
            img = cv2.cvtColor(cv2.imread(path_img), cv2.COLOR_BGR2GRAY)
            img = np.round(img / 255.,5)
            img = cv2.resize(img, (self.y_col, self.x_col))
            img = cv2.resize(img, (self.dim[1], self.dim[0]))
            img = np.expand_dims(img, axis = 2) # add the dimension of the channel 
            
            # Put it into X matrix
            X[ii,] = img
            y[ii] = labels[list_art[ii]]

        return X, y

In [29]:
x_col, y_col = (256, 937)
batch_size = 5
num_classes = df_artists.shape[0]
shuffle = False
scaling = 2

training_generator = DataGenerator(df=df_concat, 
                                   scaling = scaling,
                                   x_col=x_col, 
                                   y_col=y_col,
                                   batch_size=batch_size, 
                                   num_classes=num_artists,
                                   shuffle=False)

# Model

In [30]:
def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters

def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters

def dense_net(filters, growth_rate, classes, dense_block_size, layers_in_block, in_shape):
    input_img = Input(shape=in_shape)
    x = Conv2D(3, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = GlobalAveragePooling2D()(dense_x)

    output = Dense(classes, activation='softmax')(dense_x)

    return Model(input_img, output)

## Quick model

In [31]:
def quick_CNN(input_shape, num_classes):

    model = Sequential()
    model.add(Conv2D(2, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(4, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [32]:
def quick_CNN2(input_shape, num_classes):

    model = Sequential()
    model.add(Conv1D(16, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(4, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

# CRNN

In [33]:
def CRNN2D(X_shape, nb_classes):
    '''
    Model used for evaluation in paper. Inspired by K. Choi model in:
    https://github.com/keunwoochoi/music-auto_tagging-keras/blob/master/music_tagger_crnn.py
    '''

    nb_layers = 4  # number of convolutional layers
    nb_filters = [64, 128, 128, 128]  # filter sizes
    kernel_size = (3, 3)  # convolution kernel size
    activation = 'elu'  # activation function to use after each layer
    pool_size = [(2, 2), (4, 2), (4, 2), (4, 2),
                 (4, 2)]  # size of pooling area

    # shape of input data (frequency, time, channels)
    input_shape = X_shape
    frequency_axis = 1
    time_axis = 2
    channel_axis = 3

    # Create sequential model and normalize along frequency axis
    model = Sequential()
    model.add(BatchNormalization(axis=frequency_axis, input_shape=input_shape))

    # First convolution layer specifies shape
    model.add(Conv2D(nb_filters[0], kernel_size=kernel_size, padding='same',
                     data_format="channels_last",
                     input_shape=input_shape))
    model.add(Activation(activation))
    model.add(BatchNormalization(axis=channel_axis))
    model.add(MaxPooling2D(pool_size=pool_size[0], strides=pool_size[0]))
    model.add(Dropout(0.1))

    # Add more convolutional layers
    for layer in range(nb_layers - 1):
        # Convolutional layer
        model.add(Conv2D(nb_filters[layer + 1], kernel_size=kernel_size,
                         padding='same'))
        model.add(Activation(activation))
        model.add(BatchNormalization(
            axis=channel_axis))  # Improves overfitting/underfitting
        model.add(MaxPooling2D(pool_size=pool_size[layer + 1],
                               strides=pool_size[layer + 1]))  # Max pooling
        model.add(Dropout(0.1))

        # Reshaping input for recurrent layer
    # (frequency, time, channels) --> (time, frequency, channel)
    model.add(Permute((time_axis, frequency_axis, channel_axis)))
    resize_shape = model.output_shape[2] * model.output_shape[3]
    model.add(Reshape((model.output_shape[1], resize_shape)))

    # recurrent layer
    model.add(GRU(32, return_sequences=True))
    model.add(GRU(32, return_sequences=False))
    model.add(Dropout(0.3))

    # Output layer
    model.add(Dense(nb_classes))
    model.add(Activation("softmax"))
    return model


## Instantiate the model

In [34]:
#dense_block_size = 4
#layers_in_block = 2

#growth_rate = 2
in_shape = (int(x_col / scaling), int(y_col / scaling), 1)
classes = num_artists

#model = dense_net(growth_rate * 2, growth_rate, classes, dense_block_size, layers_in_block, in_shape)
#model = quick_CNN2(input_shape=in_shape, num_classes = classes)
model = CRNN2D(in_shape, classes)

INFO:plaidml:Opening device "metal_amd_radeon_pro_580x.0"


## Callbacks

In [35]:
save_metrics_folder='metrics'
save_weights_folder='weights'
load_checkpoint = False
early_stop=20

In [36]:
weights = os.path.join(save_weights_folder, 'crnn_v1_epoch_{epoch:02d}.hdf5')
os.makedirs(save_weights_folder, exist_ok=True)
os.makedirs(save_metrics_folder, exist_ok=True)

In [37]:
if load_checkpoint:
    print("Looking for previous weights...")
    if isfile(weights):
        print('Checkpoint file detected. Loading weights.')
        model.load_weights(weights)
    else:
        print('No checkpoint file detected.  Starting from scratch.')
else:
    print('Starting from scratch (no checkpoint)')

Starting from scratch (no checkpoint)


In [38]:
checkpointer = ModelCheckpoint(filepath=weights,
                                   verbose=1,
                                   save_best_only=False,
                                   save_weights_only=True,
                                   period=1,
                                   monitor = 'categorical_crossentropy')
earlystopper = EarlyStopping(monitor='categorical_crossentropy', min_delta=0,
                             patience=early_stop, verbose=0, mode='auto')

## Parameters

In [39]:
# training
epochs = 200
lr = 0.001
optimizer = Adam(lr=lr)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator = training_generator, 
          epochs=epochs, 
          callbacks=[checkpointer, earlystopper],
          shuffle=True)

Epoch 1/200


INFO:plaidml:Analyzing Ops: 4686 of 5476 operations complete
